In [97]:
import cv2
import easyocr
import json
from PIL import Image, ImageDraw
import re

# EasyOCR reader 객체 생성 (한글 및 영어 지원)
reader = easyocr.Reader(['ko', 'en'], gpu=False)

def process_image(img):
    # 이미지 읽기
    image = cv2.imread(img)
    if image is None:
        raise Exception(f"이미지를 읽을 수 없습니다: {img}")
    
    # 이미지에서 텍스트 추출
    results = reader.readtext(image)
    
    # 추출된 텍스트를 저장할 리스트
    extracted_text = []
    
    # 파란색 상자를 그리기 위한 PIL 이미지로 변환
    img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img)
    
    # 각 텍스트 주위에 상자를 그리고 텍스트를 저장
    for detection in results:
        bbox = detection[0]
        text = detection[1]
        extracted_text.append(text)
        
        # 상자 좌표
        x_min, y_min = int(bbox[0][0]), int(bbox[0][1])
        x_max, y_max = int(bbox[2][0]), int(bbox[2][1])
        
        # 상자 그리기
        draw.rectangle([x_min, y_min, x_max, y_max], outline="blue", width=2)
        draw.text((x_min, y_min - 20), text, fill="blue")
    
    # 정보 추출 함수들
    def extract_business_numbers(text_list):
        business_number_pattern = re.compile(r'\b\d{3}-?\d{2}-?\d{5}\b')
        return [match for text in text_list for match in business_number_pattern.findall(text)]
    
    def extract_store_names(text_list):
        # 오타를 포함한 키워드 패턴
        store_name_pattern = re.compile(r'(상호명|회사명|업체명|가맹점명|가맣점명|상오|[상싱성][호오]|[회훼]사)\s*[:;：]?\s*([^\s)]+(?:\s*\S*)*?[점]\s*?\S*)')
        return [match[1] for text in text_list for match in store_name_pattern.findall(text)]

    

    def is_valid_date(date_str):
        """날짜 형식이 유효한지 검사하는 함수"""
        parts = re.split(r'[-/]', date_str)  # '-'와 '/'를 구분자로 사용
        if len(parts) == 3:
            # MM-DD-YYYY 또는 YYYY-MM-DD 또는 YYYY/MM/DD 형식 체크
            if len(parts[0]) == 2 and len(parts[2]) == 4:  # MM-DD-YYYY
                month, day, year = map(int, parts)
                return 1 <= month <= 12 and 1 <= day <= 31 and year >= 2000 and year <= 2100
            elif len(parts[2]) == 2 and len(parts[0]) == 4:  # YYYY-MM-DD
                year, month, day = map(int, parts)
                return 1 <= month <= 12 and 1 <= day <= 31 and year >= 2000 and year <= 2100
            elif len(parts[0]) == 4 and len(parts[1]) == 2 and len(parts[2]) == 2:  # YYYY/MM/DD
                year, month, day = map(int, parts)
                return 1 <= month <= 12 and 1 <= day <= 31 and year >= 2000 and year <= 2100
        return False
    
    def extract_transaction_date(json_data):
        # 다양한 날짜 형식을 허용하는 정규식
        date_pattern = re.compile(
            r'([거기][래레][일닐]|[결겔]제[일닐]|거[래레]일|[결겔]제날짜|날짜)?\s*[:;：]?\s*'
            r'(\d{4}[-/]\d{2}[-/]\d{2}|\d{2}[-/]\d{2}[-/]\d{2})'  # MM-DD-YYYY or YYYY-MM-DD 형식
        )
    
        extracted_dates = []
    
        for text in json_data:  # json_data가 문자열 목록이라고 가정
            matches = date_pattern.findall(text)
    
            for match in matches:
                date_part = match[1]  # 날짜 부분만 추출
                
                # 유효한 날짜만 추가
                if is_valid_date(date_part):
                    extracted_dates.append(date_part)
    
        return extracted_dates



    # 영수증 6번 빼고 다 됨.
    # def extract_transaction_date(json_data):
    #     # 다양한 날짜 형식을 허용하는 정규식
    #     date_pattern = re.compile(
    #         r'([거기][래레][일닐]|[결겔]제[일닐]|거[래레]일|[결겔]제날짜|날짜)?\s*[:;：]?\s*(\d{4}[-/]\d{2}[-/]\d{2}|\d{1,2}[-/]\d{1,2}[-/]\d{2,4})'
    #     )
    
    #     extracted_dates = []
    
    #     for text in json_data:  # json_data가 문자열 목록이라고 가정
    #         matches = date_pattern.findall(text)
    
    #         for match in matches:
    #             date_part = match[1]  # 날짜 부분만 추출
                
    #             # 년도 형식 검사 (YYYY, MM-DD, DD-MM)
    #             if len(date_part.split('-')) == 3:
    #                 parts = date_part.split('-')
    #                 year = parts[-1]  # 마지막 부분이 연도
    
    #                 if len(year) == 2:  # YY 형식
    #                     year = '20' + year  # 2000년대
    
    #                 # YYYY 형식으로 변환 및 연도 체크
    #                 if len(year) == 4 and 2000 <= int(year) <= 2100:
    #                     extracted_dates.append(date_part)
    #             elif len(date_part.split('/')) == 3:  # DD/MM/YYYY 형식도 체크
    #                 day, month, year = date_part.split('/')
    #                 if 2000 <= int(year) <= 2100:  # 연도 체크
    #                     extracted_dates.append(date_part)
    
    #     return extracted_dates

    
    
    def extract_total_price(json_data):
        # 다양한 가격 형식을 허용하는 정규식
        total_price_pattern = re.compile(
            r'(총\s*가격|합계|Total|TOTAL)\s*[:;：]?\s*([₩$€]?\s?\d{1,3}(?:,\d{3})*(?:\.\d{2})?)'
        )
        
        extracted_prices = []
    
        for text in json_data:  # json_data가 문자열 목록이라고 가정
            matches = total_price_pattern.findall(text)
            
            for match in matches:
                price = match[1].replace(',', '').strip()  # 가격에서 콤마 제거
                extracted_prices.append(price)
    
        return extracted_prices


    
    # 정보 추출
    info_dict = {
        "사업자번호": extract_business_numbers(extracted_text),
        "가맹점명": extract_store_names(extracted_text),
        "거래일시": extract_transaction_date(extracted_text),
        "총 가격": extract_total_price(extracted_text)
    }
    
    return info_dict

# 이미지 처리 및 정보 추출
a = process_image('bill2.jpeg')
print(json.dumps(a, ensure_ascii=False, indent=4))


Using CPU. Note: This module is much faster with a GPU.


{
    "사업자번호": [
        "307-21-61771"
    ],
    "가맹점명": [
        "타임유 (망월 사역점)"
    ],
    "거래일시": [
        "2024-07-20"
    ],
    "총 가격": []
}
